In [1]:
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import psycopg2

In [5]:
!pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [26]:
from dotenv import load_dotenv
import os
load_dotenv()
print(os.environ.get('username'))

retail_user


In [27]:
conn = psycopg2.connect(
    host=os.getenv('host'),
    port=5432,
    database='retail_db',
    user=os.getenv('username'),
    password=os.getenv('password')
)

In [29]:
cur = conn.cursor()

In [30]:
cur.execute('SELECT * FROM orders limit 10')

In [31]:
cur.fetchall()

[(1, datetime.datetime(2013, 7, 25, 0, 0), 11599, 'CLOSED'),
 (2, datetime.datetime(2013, 7, 25, 0, 0), 256, 'PENDING_PAYMENT'),
 (3, datetime.datetime(2013, 7, 25, 0, 0), 12111, 'COMPLETE'),
 (4, datetime.datetime(2013, 7, 25, 0, 0), 8827, 'CLOSED'),
 (5, datetime.datetime(2013, 7, 25, 0, 0), 11318, 'COMPLETE'),
 (6, datetime.datetime(2013, 7, 25, 0, 0), 7130, 'COMPLETE'),
 (7, datetime.datetime(2013, 7, 25, 0, 0), 4530, 'COMPLETE'),
 (8, datetime.datetime(2013, 7, 25, 0, 0), 2911, 'PROCESSING'),
 (9, datetime.datetime(2013, 7, 25, 0, 0), 5657, 'PENDING_PAYMENT'),
 (10, datetime.datetime(2013, 7, 25, 0, 0), 5648, 'PENDING_PAYMENT')]

In [32]:
cur.execute('''
  SELECT order_status,
            count(1) AS order_count
  FROM orders
  GROUP BY 1
  ORDER BY 2 DESC
''')
cur.fetchall()

[('COMPLETE', 22899),
 ('PENDING_PAYMENT', 15030),
 ('PROCESSING', 8275),
 ('PENDING', 7610),
 ('CLOSED', 7556),
 ('ON_HOLD', 3798),
 ('SUSPECTED_FRAUD', 1558),
 ('CANCELED', 1428),
 ('PAYMENT_REVIEW', 729)]

### Postgres with Pandas

In [33]:
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.4/241.4 kB 9.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [34]:
import pandas as pd

In [35]:
column = ['order_id', 'order_date', 'order_customer_id', 'order_status']

In [37]:
orders = pd.read_csv('../../data/retail_db/orders/part-00000', names=column)
orders.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


In [38]:
import json

In [39]:
schemas = json.load(open('../../data/retail_db/schemas.json'))

In [42]:
sorted(schemas['orders'], key=lambda col:col['column_position'])

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [43]:
columns = [col['column_name'] for col in sorted(schemas['orders'], key=lambda col:col['column_position'])]
columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [44]:
orders = pd.read_csv('../../data/retail_db/orders/part-00000', names=columns)
orders.head()

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE


In [45]:
type(orders)

pandas.core.frame.DataFrame

In [46]:
orders

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD


In [47]:
orders.shape

(68883, 4)

In [48]:
orders.groupby(['order_date', 'order_status'])['order_id'].count()

order_date             order_status   
2013-07-25 00:00:00.0  CANCELED            1
                       CLOSED             20
                       COMPLETE           42
                       ON_HOLD             5
                       PAYMENT_REVIEW      3
                                          ..
2014-07-24 00:00:00.0  PAYMENT_REVIEW      2
                       PENDING            23
                       PENDING_PAYMENT    37
                       PROCESSING         20
                       SUSPECTED_FRAUD     4
Name: order_id, Length: 3203, dtype: int64

In [55]:
daily_status_count = orders.groupby(['order_date', 'order_status'])['order_id']. \
  agg(order_count='count'). \
  reset_index()

In [56]:
daily_status_count

,order_date,order_status,order_count
0,2013-07-25 00:00:00.0,CANCELED,1
1,2013-07-25 00:00:00.0,CLOSED,20
2,2013-07-25 00:00:00.0,COMPLETE,42
3,2013-07-25 00:00:00.0,ON_HOLD,5
4,2013-07-25 00:00:00.0,PAYMENT_REVIEW,3
...,...,...,...
3198,2014-07-24 00:00:00.0,PAYMENT_REVIEW,2
3199,2014-07-24 00:00:00.0,PENDING,23
3200,2014-07-24 00:00:00.0,PENDING_PAYMENT,37
3201,2014-07-24 00:00:00.0,PROCESSING,20


In [57]:
daily_status_count.to_sql?

Signature:
daily_status_count.to_sql(
    name: 'str',
    con,
    schema: 'str | None' = None,
    if_exists: "Literal['fail', 'replace', 'append']" = 'fail',
    index: 'bool_t' = True,
    index_label: 'IndexLabel' = None,
    chunksize: 'int | None' = None,
    dtype: 'DtypeArg | None' = None,
    method: 'str | None' = None,
) -> 'int | None'
Docstring:
Write records stored in a DataFrame to a SQL database.

Databases supported by SQLAlchemy [1]_ are supported. Tables can be
newly created, appended to, or overwritten.

Parameters
----------
name : str
    Name of SQL table.
con : sqlalchemy.engine.(Engine or Connection) or sqlite3.Connection
    Using SQLAlchemy makes it possible to use any DB supported by that
    library. Legacy support is provided for sqlite3.Connection objects. The user
    is responsible for engine disposal and connection closure for the SQLAlchemy
    connectable. See `here                 <https://docs.sqlalchemy.org/en/20/core/connections.html>`_.
    If 

In [61]:
host=os.getenv('host')
port=os.getenv('port')
database='retail_db'
user=os.getenv('username')
password=os.getenv('password')

In [62]:
conn_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [65]:
daily_status_count.to_sql(
    'daily_status_count',
    conn_uri,
    if_exists='replace',
    index=False
)

203

In [66]:
pd.read_sql?

Signature:
pd.read_sql(
    sql,
    con,
    index_col: 'str | list[str] | None' = None,
    coerce_float: 'bool' = True,
    params=None,
    parse_dates=None,
    columns: 'list[str] | None' = None,
    chunksize: 'int | None' = None,
    dtype_backend: 'DtypeBackend | lib.NoDefault' = <no_default>,
    dtype: 'DtypeArg | None' = None,
) -> 'DataFrame | Iterator[DataFrame]'
Docstring:
Read SQL query or database table into a DataFrame.

This function is a convenience wrapper around ``read_sql_table`` and
``read_sql_query`` (for backward compatibility). It will delegate
to the specific function depending on the provided input. A SQL query
will be routed to ``read_sql_query``, while a database table name will
be routed to ``read_sql_table``. Note that the delegated function might
have more specific notes about their functionality not listed here.

Parameters
----------
sql : str or SQLAlchemy Selectable (select or text object)
    SQL query to be executed or a table name.
con : SQLAlch

In [69]:
dsc = pd.read_sql(
    'daily_status_count',
    conn_uri
)

In [70]:
dsc.shape

(3203, 3)

In [71]:
dsc.describe()

,order_count
count,3203.000000
mean,21.505776
std,20.909479
min,1.000000
25%,5.000000
50%,15.000000
75%,30.000000
max,118.000000


In [72]:
dsc = pd.read_sql(
    '''
    SELECT order_status, count(*) AS order_count FROM daily_status_count
    GROUP BY 1
    ORDER BY 2 DESC
    ''',

    conn_uri
)

In [73]:
dsc

,order_status,order_count
0,COMPLETE,364
1,ON_HOLD,364
2,PENDING_PAYMENT,364
3,PENDING,364
4,CLOSED,364
5,PROCESSING,364
6,CANCELED,354
7,SUSPECTED_FRAUD,352
8,PAYMENT_REVIEW,313


In [74]:
!pip install google-cloud-secret-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 14.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
